In [125]:
import tensorflow as tf
import cv2
import os
import matplotlib.pyplot as plt
import numpy as np
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.applications import MobileNetV3Large

In [126]:
#directory and classes
Datadirectory = r"C:\Users\Auni\Documents\SEM8\SECJ4134_SOFTWARE_ENGINEERING_PROJECT_II\FER2013\train"

In [127]:
Classes = ["angry", "disgust", "fear", "happy", "neutral", "sad", "surprise"]

In [128]:
img_size = 224

In [132]:
# Function to load and preprocess images
def load_and_preprocess_image(image_path, label):
    # Read the image file
    img = tf.io.read_file(image_path)
    
    # Decode the image as an RGB image
    img = tf.image.decode_image(img, channels=3, expand_animations=False)
    
    # Resize the image to the target size
    img = tf.image.resize(img, [img_size, img_size])
    
    # Normalize the image to [0, 1]
    img = img / 255.0
    
    return img, label

In [133]:
# Function to create the tf.data.Dataset
def create_dataset(data_dir, classes):
    image_paths = []
    labels = []
    for class_name in classes:
        class_dir = os.path.join(data_dir, class_name)
        for img_name in os.listdir(class_dir):
            img_path = os.path.join(class_dir, img_name)

            # Check if it's a file and has a valid image extension
            if os.path.isfile(img_path) and img_name.lower().endswith(('.jpg', '.jpeg', '.png')):
                image_paths.append(img_path)
                labels.append(classes.index(class_name))

    # Convert to TensorFlow tensors
    image_paths = tf.constant(image_paths)
    labels = tf.constant(labels)

    # Create a TensorFlow dataset
    dataset = tf.data.Dataset.from_tensor_slices((image_paths, labels))

    # Apply preprocessing
    dataset = dataset.map(lambda image_path, label: load_and_preprocess_image(image_path, label), num_parallel_calls=tf.data.AUTOTUNE)

    return dataset

In [134]:
# Create the dataset
dataset = create_dataset(Datadirectory, Classes)
dataset = dataset.batch(32)  # Batch the dataset
dataset = dataset.prefetch(tf.data.AUTOTUNE)  # Prefetch for performance

In [135]:
#model building
input_layer = keras.Input(shape=(img_size, img_size, 3))

In [136]:
# Inspect dataset structure
for images, labels in dataset.take(1):
    print("Images shape:", images.shape)  # Should be (32, 224, 224, 3)
    print("Labels shape:", labels.shape)  # Should be (32,)

Images shape: (32, 224, 224, 3)
Labels shape: (32,)


In [137]:
input_layer.shape

(None, 224, 224, 3)

In [138]:
model = MobileNetV3Large(weights='imagenet', input_tensor=input_layer, include_top=False)

In [139]:
model.summary()

Model: "MobileNetV3Large"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer_4       │ (None, 224, 224,  │          0 │ -                 │
│ (InputLayer)        │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ rescaling_4         │ (None, 224, 224,  │          0 │ input_layer_4[0]… │
│ (Rescaling)         │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv (Conv2D)       │ (None, 112, 112,  │        432 │ rescaling_4[0][0] │
│                     │ 16)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv_bn             │ (None, 112, 112,  │         64 │ conv[0][0]        │
│ (BatchNormalizatio… │ 16)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ activation_88       │ (None, 112, 112,  │          0 │ conv_bn[0][0]     │
│ (Activation)        │ 16)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ expanded_conv_dept… │ (None, 112, 112,  │        144 │ activation_88[0]… │
│ (DepthwiseConv2D)   │ 16)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ expanded_conv_dept… │ (None, 112, 112,  │         64 │ expanded_conv_de… │
│ (BatchNormalizatio… │ 16)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ re_lu_76 (ReLU)     │ (None, 112, 112,  │          0 │ expanded_conv_de… │
│                     │ 16)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ expanded_conv_proj… │ (None, 112, 112,  │        256 │ re_lu_76[0][0]    │
│ (Conv2D)            │ 16)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ expanded_conv_proj… │ (None, 112, 112,  │         64 │ expanded_conv_pr… │
│ (BatchNormalizatio… │ 16)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ expanded_conv_add   │ (None, 112, 112,  │          0 │ activation_88[0]… │
│ (Add)               │ 16)               │            │ expanded_conv_pr… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ expanded_conv_1_ex… │ (None, 112, 112,  │      1,024 │ expanded_conv_ad… │
│ (Conv2D)            │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ expanded_conv_1_ex… │ (None, 112, 112,  │        256 │ expanded_conv_1_… │
│ (BatchNormalizatio… │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ re_lu_77 (ReLU)     │ (None, 112, 112,  │          0 │ expanded_conv_1_… │
│                     │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ expanded_conv_1_de… │ (None, 113, 113,  │          0 │ re_lu_77[0][0]    │
│ (ZeroPadding2D)     │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ expanded_conv_1_de… │ (None, 56, 56,    │        576 │ expanded_conv_1_… │
│ (DepthwiseConv2D)   │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ expanded_conv_1_de… │ (None, 56, 56,    │        256 │ expanded_conv_1_

 Total params: 2,996,352 (11.43 MB)

 Trainable params: 2,971,952 (11.34 MB)

 Non-trainable params: 24,400 (95.31 KB)

In [140]:
base_input = model.input

In [141]:
base_output = model.output

In [142]:
base_output = layers.GlobalAveragePooling2D()(base_output)

In [143]:
base_output

<KerasTensor shape=(None, 960), dtype=float32, sparse=False, ragged=False, name=keras_tensor_1039>

In [144]:
final_output = layers.Dense(128)(base_output)
final_output = layers.Activation('relu')(final_output)
final_output = layers.Dense(64)(final_output)
final_output = layers.Activation('relu')(final_output)
final_output = layers.Dense(7, activation='softmax')(final_output)

In [145]:
final_output

<KerasTensor shape=(None, 7), dtype=float32, sparse=False, ragged=False, name=keras_tensor_1044>

In [146]:
new_model = keras.Model(inputs=base_input, outputs=final_output)

In [147]:
new_model.summary()

Model: "functional_4"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer_4       │ (None, 224, 224,  │          0 │ -                 │
│ (InputLayer)        │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ rescaling_4         │ (None, 224, 224,  │          0 │ input_layer_4[0]… │
│ (Rescaling)         │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv (Conv2D)       │ (None, 112, 112,  │        432 │ rescaling_4[0][0] │
│                     │ 16)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv_bn             │ (None, 112, 112,  │         64 │ conv[0][0]        │
│ (BatchNormalizatio… │ 16)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ activation_88       │ (None, 112, 112,  │          0 │ conv_bn[0][0]     │
│ (Activation)        │ 16)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ expanded_conv_dept… │ (None, 112, 112,  │        144 │ activation_88[0]… │
│ (DepthwiseConv2D)   │ 16)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ expanded_conv_dept… │ (None, 112, 112,  │         64 │ expanded_conv_de… │
│ (BatchNormalizatio… │ 16)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ re_lu_76 (ReLU)     │ (None, 112, 112,  │          0 │ expanded_conv_de… │
│                     │ 16)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ expanded_conv_proj… │ (None, 112, 112,  │        256 │ re_lu_76[0][0]    │
│ (Conv2D)            │ 16)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ expanded_conv_proj… │ (None, 112, 112,  │         64 │ expanded_conv_pr… │
│ (BatchNormalizatio… │ 16)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ expanded_conv_add   │ (None, 112, 112,  │          0 │ activation_88[0]… │
│ (Add)               │ 16)               │            │ expanded_conv_pr… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ expanded_conv_1_ex… │ (None, 112, 112,  │      1,024 │ expanded_conv_ad… │
│ (Conv2D)            │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ expanded_conv_1_ex… │ (None, 112, 112,  │        256 │ expanded_conv_1_… │
│ (BatchNormalizatio… │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ re_lu_77 (ReLU)     │ (None, 112, 112,  │          0 │ expanded_conv_1_… │
│                     │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ expanded_conv_1_de… │ (None, 113, 113,  │          0 │ re_lu_77[0][0]    │
│ (ZeroPadding2D)     │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ expanded_conv_1_de… │ (None, 56, 56,    │        576 │ expanded_conv_1_… │
│ (DepthwiseConv2D)   │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ expanded_conv_1_de… │ (None, 56, 56,    │        256 │ expanded_conv_1_

 Total params: 3,128,071 (11.93 MB)

 Trainable params: 3,103,671 (11.84 MB)

 Non-trainable params: 24,400 (95.31 KB)

In [148]:
# Inspect dataset structure
for element in dataset.take(1):
    print(element)

(<tf.Tensor: shape=(32, 224, 224, 3), dtype=float32, numpy=
array([[[[0.19215687, 0.19215687, 0.19215687],
         [0.19215687, 0.19215687, 0.19215687],
         [0.1894958 , 0.1894958 , 0.1894958 ],
         ...,
         [0.3619045 , 0.3619045 , 0.3619045 ],
         [0.3529412 , 0.3529412 , 0.3529412 ],
         [0.3529412 , 0.3529412 , 0.3529412 ]],

        [[0.19215687, 0.19215687, 0.19215687],
         [0.19215687, 0.19215687, 0.19215687],
         [0.1894958 , 0.1894958 , 0.1894958 ],
         ...,
         [0.3619045 , 0.3619045 , 0.3619045 ],
         [0.3529412 , 0.3529412 , 0.3529412 ],
         [0.3529412 , 0.3529412 , 0.3529412 ]],

        [[0.19313726, 0.19313726, 0.19313726],
         [0.19313726, 0.19313726, 0.19313726],
         [0.1904562 , 0.1904562 , 0.1904562 ],
         ...,
         [0.3621746 , 0.3621746 , 0.3621746 ],
         [0.35322127, 0.35322127, 0.35322127],
         [0.35322127, 0.35322127, 0.35322127]],

        ...,

        [[0.39663863, 0.39663863

In [149]:
# Compile and train the model
new_model.compile(loss="sparse_categorical_crossentropy", optimizer="adam", metrics=["accuracy"])

In [151]:
history = new_model.fit(dataset, epochs=15)

Epoch 1/15
898/898 ━━━━━━━━━━━━━━━━━━━━ 696s 775ms/step - accuracy: 0.8817 - loss: 0.5764
Epoch 2/15
898/898 ━━━━━━━━━━━━━━━━━━━━ 694s 773ms/step - accuracy: 0.8646 - loss: 0.5653
Epoch 3/15
898/898 ━━━━━━━━━━━━━━━━━━━━ 693s 772ms/step - accuracy: 0.8612 - loss: 0.7220
Epoch 4/15
898/898 ━━━━━━━━━━━━━━━━━━━━ 692s 771ms/step - accuracy: 0.8962 - loss: 0.4903
Epoch 5/15
898/898 ━━━━━━━━━━━━━━━━━━━━ 696s 775ms/step - accuracy: 0.9153 - loss: 0.4630
Epoch 6/15
898/898 ━━━━━━━━━━━━━━━━━━━━ 715s 796ms/step - accuracy: 0.9155 - loss: 0.5357
Epoch 7/15
898/898 ━━━━━━━━━━━━━━━━━━━━ 712s 793ms/step - accuracy: 0.8895 - loss: 0.5324
Epoch 8/15
898/898 ━━━━━━━━━━━━━━━━━━━━ 713s 793ms/step - accuracy: 0.9153 - loss: 0.5155
Epoch 9/15
898/898 ━━━━━━━━━━━━━━━━━━━━ 737s 821ms/step - accuracy: 0.8838 - loss: 0.6502
Epoch 10/15
898/898 ━━━━━━━━━━━━━━━━━━━━ 739s 823ms/step - accuracy: 0.8806 - loss: 0.5750
Epoch 11/15
898/898 ━━━━━━━━━━━━━━━━━━━━ 722s 804ms/step - accuracy: 0.8851 - loss: 0.5215
Epoch 12

In [152]:
print("\nTraining History:")
for epoch in range(15):
    print(f"Epoch {epoch + 1}/15 - loss: {history.history['loss'][epoch]:.4f} - accuracy: {history.history['accuracy'][epoch]:.4f}")


Training History:
Epoch 1/15 - loss: 0.3529 - accuracy: 0.9322
Epoch 2/15 - loss: 0.4405 - accuracy: 0.8865
Epoch 3/15 - loss: 0.4077 - accuracy: 0.9127
Epoch 4/15 - loss: 0.3432 - accuracy: 0.9208
Epoch 5/15 - loss: 0.2940 - accuracy: 0.9431
Epoch 6/15 - loss: 0.3440 - accuracy: 0.9400
Epoch 7/15 - loss: 0.3214 - accuracy: 0.9319
Epoch 8/15 - loss: 0.3933 - accuracy: 0.9412
Epoch 9/15 - loss: 0.3653 - accuracy: 0.9343
Epoch 10/15 - loss: 0.3471 - accuracy: 0.9352
Epoch 11/15 - loss: 0.3670 - accuracy: 0.9265
Epoch 12/15 - loss: 0.4540 - accuracy: 0.9139
Epoch 13/15 - loss: 0.3824 - accuracy: 0.9013
Epoch 14/15 - loss: 0.3250 - accuracy: 0.9204
Epoch 15/15 - loss: 0.4195 - accuracy: 0.9039


# 